# Kasus 1 : Persamaan Sederahana
Mencari solusi dari sebuah Persamaan sederahana contoh:

> $a + 2b - 3c +4d = 30$

mentukan nilai $a, b, c, d$

## Populasi Awal

Representasi yang digunakan : $Real  Number$

Jumlah gen pada setiap kromosom akan tergantung persamaan yang dimasukkan 

Dalam setiap satu populasi pada sebuah generasi akan terdiri dari 10 Kromosom


In [0]:
import random
import re
import operator as operan

Inputkan persamaan dibawah, batasan :
*   Constanta hanya terdiri dari 1 Digit
*   Operator yang di kenali hanya tambah(+) dan kurang (-)



In [0]:
equation = 'a+2b=30'

In [0]:
Genes = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10]
ops = { "+": operan.add, 
        "-": operan.sub }

In [0]:
def break_equation(equation):
  equation_resul = int(equation.split('=')[1])

  equation_body = equation.split('=')[0]
  equation_split_op = re.split(r'[-+]', equation_body)
  geneSize = len(equation_split_op)

  operator = []
  for x in list(equation_body):
    if (x == '-' or x == '+'):
      operator.append(x)

  cons = []
  for y in equation_split_op:
    if len(list(y)) == 1:
      cons.append(1)
    else:
      cons.append(int(list(y)[0]))
  
  return cons, operator, equation_resul, geneSize

In [0]:
cons, operator, equation_resul, geneSize = break_equation(equation)
print(cons)
print(operator)
print(equation_resul)
print(geneSize)


[1, 2]
['+']
30
2


In [0]:
def rand_gene(Genes): 
  gene = random.choice(Genes) 
  return gene

def create_chromosome():
  return [rand_gene(Genes) for i in range(geneSize)]

In [0]:
class Individual:
  def __init__(self, chromosome):
    self.chromosome = chromosome
    self.fitness = self.cal_fitness()

  global cons
  global operator
  global equation_resul
  global ops
  
  def cal_fitness(self):
    result = ops[operator[0]]((cons[0]*self.chromosome[0]),(cons[1]*self.chromosome[1]))
    for i in range(1, len(operator)):
      result = ops[operator[i]](result,(cons[i+1]*self.chromosome[i+1]))

    minimasi = abs(equation_resul-result)
    
    return minimasi;

In [0]:
PopulationSize = 10
population = []

# create initial population
for i in range(PopulationSize): 
  chromosome = create_chromosome()
  population.append(Individual(chromosome))

# population = sorted(population, key = lambda x:x.fitness)
print('Init Generation:\n')
for i in range(PopulationSize): 
  print("Individu {}: {} - Fit: {}".format(i+1,population[i].chromosome, population[i].fitness)) 


Init Generation:

Individu 1: [3, -2] - Fit: 31
Individu 2: [-8, -8] - Fit: 54
Individu 3: [4, 10] - Fit: 6
Individu 4: [2, 1] - Fit: 26
Individu 5: [10, 5] - Fit: 10
Individu 6: [10, -9] - Fit: 38
Individu 7: [6, 8] - Fit: 8
Individu 8: [-4, 7] - Fit: 20
Individu 9: [-8, -10] - Fit: 58
Individu 10: [-1, -8] - Fit: 47


## Proses Evolusi

In [0]:
class Selection:
    @staticmethod
    def roulette_wheel_selection(pop):
        """Roulette Wheel Parent Selection.

        Args:
            population: List of chromosomes.

        Returns:
            both of selected chromosomes.

        """
    # compute sum of fitness
        sum_of_fitness = 0
        for chromosome in pop:
            sum_of_fitness += chromosome.fitness

    # generate random number
        random_number = [random.uniform(0,sum_of_fitness), random.uniform(0,sum_of_fitness)]

    # get first selected chromosome
        sum_of_fitness = 0
        no_of_selected_chromosome = [0, 0]
        for chromosome in pop:
            sum_of_fitness += chromosome.fitness
            if sum_of_fitness > random_number[0]:
                break
            else:
                no_of_selected_chromosome[0] += 1

    # get second selected chromosome            
        sum_of_fitness = 0
        for chromosome in pop:
            sum_of_fitness += chromosome.fitness
            if sum_of_fitness > random_number[1]:
                break
            else:
                no_of_selected_chromosome[1] += 1
    
    # check if both of selected chromosomes are same
        if no_of_selected_chromosome[0] == no_of_selected_chromosome[1]:
            sum_of_fitness = 0
            no_of_selected_chromosome[1] = 0
            
            for chromosome in pop:
                sum_of_fitness += chromosome.fitness
                if sum_of_fitness > random_number[1]:
                    break
                else:
                    no_of_selected_chromosome[1] += 1

        return pop[no_of_selected_chromosome[0]], pop[no_of_selected_chromosome[1]]
    
    @staticmethod
    def rank_selection(pop):
        """Rank Parent Selection.

        Args:
            population: List of chromosomes.

        Returns:
            both of selected chromosomes.

        """
    # set size buffer population
        buffer_size = 4
        
    # generate random number
        random_number = random.sample(range(0,PopulationSize-1),buffer_size)
        
    # create buffer population from population with generated random numbers
        buffer_population = []
        for num in random_number:
            buffer_population.append(pop[num])
        
    # select best of 2 parent candidate from buffer population
        selected_parent_1 = min(buffer_population, key=lambda x: x.fitness)
        buffer_population.remove(min(buffer_population, key=lambda x: x.fitness))
        selected_parent_2 = min(buffer_population, key=lambda x: x.fitness)

        return selected_parent_1, selected_parent_2

class Crossover:
    @staticmethod
    def uniform_crossover(parent_1, parent_2):
        """Uniform Crossover.

        Args:
            population: List of chromosomes.

        Returns:
            chromosome of offspring.

        """
    # set threshold
        threshold = 0.5

    # generate random number
        random_number = [random.random() for i in range(geneSize)]

    # crossovering
        buffer_of_offspring = [0 for i in range(geneSize)]
        for no_of_gen in range(geneSize):
            if random_number[no_of_gen] > threshold:
                buffer_of_offspring[no_of_gen] = parent_1.chromosome[no_of_gen]
            else:
                buffer_of_offspring[no_of_gen] = parent_2.chromosome[no_of_gen]

        return buffer_of_offspring
    
    @staticmethod
    def arithmetic_crossover(parent_1, parent_2):
        """Arithmetic Crossover.

        Args:
            population: List of chromosomes.

        Returns:
            chromosome of offspring.

        """

    # crossovering
        buffer_of_offspring = [0 for i in range(geneSize)]
        for no_of_gen in range(geneSize):
                buffer_of_offspring[no_of_gen] = Genes[int(parent_1.chromosome[no_of_gen] + parent_2.chromosome[no_of_gen]) % len(Genes)]

        return buffer_of_offspring

class Mutation:
    @staticmethod
    def mutate(offspring):
        """Mutation of offspring

        Args:
            population: List of chromosomes.

        Returns:
            chromosome of offspring.

        """
    # generate random number
        random_number = [random.random() for i in range(geneSize)]

    # mutating gen
        for no_of_gen in range(geneSize):
            if random_number[no_of_gen] > 0.95:
                offspring[no_of_gen] = round(offspring[no_of_gen] / 2)
        return offspring

### 1. Roulate + Uniform


In [0]:
# copy initial population
buffer_of_population = []
number_of_crossover = 5

for pop in population:
    buffer_of_population.append(Individual(pop.chromosome))

for generation in range(50):
    for i in range(number_of_crossover):
        offspring = []
# selection of parent    
        parent_1, parent_2 = Selection.roulette_wheel_selection(buffer_of_population)
    
# crossovering parent
        offspring = Crossover.uniform_crossover(parent_1, parent_2)
    
# mutating offspring
        offspring = Mutation.mutate(offspring)

# add offspring to population
        buffer_of_population.append(Individual(offspring))
    
# remove the worst fitness
    for i in range(number_of_crossover):
        buffer_of_population.remove(max(buffer_of_population, key=lambda x: x.fitness))
    print("Generation: {} \t=> Best Chromosome: {} \t with fitness: {}".format(generation + 1, min(buffer_of_population, key=lambda x: x.fitness).chromosome, min(buffer_of_population, key=lambda x: x.fitness).fitness))

Generation: 1 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 2 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 3 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 4 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 5 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 6 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 7 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 8 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 9 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 10 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 11 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 12 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 13 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 14 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 15 	=> Best Chromosome: [10, 8] 	 with fitness: 4
Generation: 16 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 17 	=

### 2. Roulate + Aritmatik


In [0]:
# copy initial population
buffer_of_population = []
number_of_crossover = 5

for pop in population:
    buffer_of_population.append(Individual(pop.chromosome))
    
for generation in range(50):
    for i in range(number_of_crossover):
# selection of parent    
        parent_1, parent_2 = Selection.roulette_wheel_selection(buffer_of_population)
    
# crossovering parent
        offspring = Crossover.arithmetic_crossover(parent_1, parent_2)
    
# mutating offspring
        offspring = Mutation.mutate(offspring)

# add offspring to population
        buffer_of_population.append(Individual(offspring))
    
# remove the worst fitness
    for i in range(number_of_crossover):
        buffer_of_population.remove(max(buffer_of_population, key=lambda x: x.fitness))
    print("Generation: {} \t=> Best Chromosome: {} \t with fitness: {}".format(generation + 1, min(buffer_of_population, key=lambda x: x.fitness).chromosome, min(buffer_of_population, key=lambda x: x.fitness).fitness))

Generation: 1 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 2 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 3 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 4 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 5 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 6 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 7 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 8 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 9 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 10 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 11 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 12 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 13 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 14 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 15 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 16 	=> Best Chromosome: [7, 10] 	 with fitness: 3
Generation: 17 	=

### 3. Rank Based + Uniform


In [0]:
# # copy initial population
buffer_of_population = []
number_of_crossover = 5

for pop in population:
    buffer_of_population.append(Individual(pop.chromosome))
    
for generation in range(50):
    for i in range(number_of_crossover):
# selection of parent    
      parent_1, parent_2 = Selection.rank_selection(buffer_of_population)

# crossovering parent
      offspring = Crossover.uniform_crossover(parent_1, parent_2)
    
# mutating offspring
      offspring = Mutation.mutate(offspring)

# add offspring to population
      buffer_of_population.append(Individual(offspring))
    
# remove the worst fitness
    for i in range(number_of_crossover):
      buffer_of_population.remove(max(buffer_of_population, key=lambda x: x.fitness))
    print("Generation: {} \t=> Best Chromosome: {} \t with fitness: {}".format(generation + 1, min(buffer_of_population, key=lambda x: x.fitness).chromosome, min(buffer_of_population, key=lambda x: x.fitness).fitness))

Generation: 1 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 2 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 3 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 4 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 5 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 6 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 7 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 8 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 9 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 10 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 11 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 12 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 13 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 14 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 15 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 16 	=> Best Chromosome: [6, 10] 	 with fitness: 4
Generation: 17 	=

### 4. Rank Based + Aritmatik


In [0]:
# copy initial population
buffer_of_population = []
number_of_crossover = 5

for pop in population:
    buffer_of_population.append(Individual(pop.chromosome))
    
for generation in range(50):
    for i in range(number_of_crossover):
# selection of parent    
      parent_1, parent_2 = Selection.rank_selection(buffer_of_population)

# crossovering parent
      offspring = Crossover.arithmetic_crossover(parent_1, parent_2)
    
# mutating offspring
      offspring = Mutation.mutate(offspring)

# add offspring to population
      buffer_of_population.append(Individual(offspring))
    
# remove the worst fitness
    for i in range(number_of_crossover):
      buffer_of_population.remove(max(buffer_of_population, key=lambda x: x.fitness))
    print("Generation: {} \t=> Best Chromosome: {} \t with fitness: {}".format(generation + 1, min(buffer_of_population, key=lambda x: x.fitness).chromosome, min(buffer_of_population, key=lambda x: x.fitness).fitness))

Generation: 1 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 2 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 3 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 4 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 5 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 6 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 7 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 8 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 9 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 10 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 11 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 12 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 13 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 14 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 15 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 16 	=> Best Chromosome: [4, 10] 	 with fitness: 6
Generation: 17 	=